### Prepare Dataset

In [1]:
def load_pos_data(path):
    dataset = []
    with open(path, "r", encoding="utf-8") as f:
        tokens = []
        tags = []
        for line in f:
            line = line.strip()
            if not line:
                # End of a sentence
                if tokens:
                    dataset.append({"tokens": tokens, "tags": tags})
                    tokens = []
                    tags = []
            else:
                parts = line.split()
                if len(parts) == 2:
                    word, tag = parts
                    tokens.append(word)
                    tags.append(tag)
        # Catch the last sentence if no newline at EOF
        if tokens:
            dataset.append({"tokens": tokens, "tags": tags})
    return dataset


In [2]:
print(load_pos_data("sinhala_pos.txt")[0:2])

[{'tokens': ['ඊශ්රායල්', 'මිසයිල', 'ප්රහාර', 'වලින්', 'පලස්තීනුවෝ', '4', 'ක්', 'මිය', 'යති', '.'], 'tags': ['NNP', 'NNJ', 'NNC', 'CM', 'NNP', 'NUM', 'RP', 'RRPCV', 'VFM', 'FS']}, {'tokens': ['ගාසා', 'තීරයේදී', '.'], 'tags': ['NNP', 'NNP', 'FS']}]


In [3]:
from datasets import Dataset, DatasetDict
import random

# all_data = load_pos_data("sinhala_pos.txt")
# random.shuffle(all_data)

# # Optional: 80% train, 20% test split
# split_idx = int(0.8 * len(all_data))
# train_data = all_data[:split_idx]
# test_data = all_data[split_idx:]

# dataset = DatasetDict({
#     "train": Dataset.from_list(train_data),
#     "test": Dataset.from_list(test_data),
# })

data = load_pos_data("sinhala_pos.txt")

dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.2)


d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### tag2id mapping

In [4]:
unique_tags = set(tag for example in data for tag in example["tags"])

tag2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
label_list = list(tag2id.keys())

id2tag = {i: tag for tag, i in tag2id.items()}


### Tokenize and Align Labels

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_and_align_labels(example):
    tokenized = tokenizer(
        example["tokens"], 
        is_split_into_words=True, 
        truncation=True, 
        padding="max_length",      # Pad to max length of the model or your max_length param
        max_length=256,            # or any max_length you want (optional)
        return_tensors=None        # don't convert to tensors here; Trainer does it later
    )
    
    word_ids = tokenized.word_ids()
    labels = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(tag2id[example["tags"][word_idx]])
        else:
            # Label only the first sub-token
            labels.append(-100)
        previous_word_idx = word_idx
    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)


Map: 100%|██████████| 2261/2261 [00:02<00:00, 1042.31 examples/s]


### Define the Model

In [6]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=len(tag2id),
    id2label=id2tag,
    label2id=tag2id,
    local_files_only=True
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Train the Model

In [7]:
from transformers import TrainingArguments, Trainer
import numpy as np
# from seqeval.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=2)
#     labels = p.label_ids

#     true_preds = [
#         [id2tag[p] for (p, l) in zip(pred_seq, label_seq) if l != -100]
#         for pred_seq, label_seq in zip(preds, labels)
#     ]
#     true_labels = [
#         [id2tag[l] for (p, l) in zip(pred_seq, label_seq) if l != -100]
#         for pred_seq, label_seq in zip(preds, labels)
#     ]

#     return {
#         "accuracy": accuracy_score(true_labels, true_preds),
#         "report": classification_report(true_labels, true_preds),
#     }

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=2)
    labels = p.label_ids

    # Convert IDs to tag names, skip padding (-100)

    
    true_preds = [
        id2tag[p] for pred_seq, label_seq in zip(preds, labels)
        for p, l in zip(pred_seq, label_seq) if l != -100
    ]
    true_labels = [
        id2tag[l] for pred_seq, label_seq in zip(preds, labels)
        for p, l in zip(pred_seq, label_seq) if l != -100
    ]

    # Optional: print detailed report to console (not return)
    print(classification_report(true_labels, true_preds, digits=4))

    return {
        "accuracy": accuracy_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds, average="weighted"),
        # Avoid printing report here in returned dict — it's too long
        # Use print manually if needed:
        # print(classification_report(true_labels, true_preds))
    }

training_args = TrainingArguments(
    output_dir="./pos-xlm-r",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_25372\1272681515.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.691000,0.317093,0.911738,0.911807
2,0.309000,0.283806,0.920654,0.921113
3,0.260500,0.275995,0.923030,0.923303
4,0.229400,0.272975,0.923538,0.923902


d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

              precision    recall  f1-score   support

         ABB     0.9622    0.9745    0.9683       470
         AUX     0.9785    0.9695    0.9740       328
          CC     0.9743    0.9639    0.9691       748
          CM     0.9545    0.9481    0.9513       443
         DET     0.9704    0.9704    0.9704      1147
          FS     1.0000    0.9996    0.9998      2256
         JCV     0.6915    0.7365    0.7133       630
          JJ     0.8674    0.8256    0.8460      3652
         NCV     0.7721    0.8462    0.8074       949
         NDT     0.0000    0.0000    0.0000        15
         NIP     0.9643    0.9774    0.9708       884
         NNC     0.9235    0.8963    0.9097     12249
         NNJ     0.6636    0.7877    0.7204      1300
         NNP     0.9163    0.9311    0.9236      4806
         NUM     0.9549    0.9429    0.9489      1033
         NVB     0.5833    0.7226    0.6455       155
         NVF     0.0000    0.0000    0.0000         1
        POST     0.9295    

d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

...

d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

              precision    recall  f1-score   support

         ABB     0.9705    0.9809    0.9757       470
         AUX     0.9876    0.9726    0.9800       328
          CC     0.9772    0.9733    0.9752       748
          CM     0.9594    0.9594    0.9594       443
         DET     0.9632    0.9817    0.9724      1147
          FS     1.0000    0.9996    0.9998      2256
         JCV     0.6953    0.7968    0.7426       630
          JJ     0.8756    0.8478    0.8614      3652
         NCV     0.8295    0.8303    0.8299       949
         NDT     0.6154    0.5333    0.5714        15
         NIP     0.9622    0.9796    0.9709       884
         NNC     0.9329    0.9091    0.9208     12249
         NNJ     0.7238    0.7923    0.7565      1300
         NNP     0.9344    0.9422    0.9383      4806
         NUM     0.9416    0.9681    0.9547      1033
         NVB     0.6283    0.7742    0.6936       155
         NVF     0.0000    0.0000    0.0000         1
        POST     0.9381    

d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

              precision    recall  f1-score   support

         ABB     0.9705    0.9809    0.9757       470
         AUX     0.9846    0.9726    0.9785       328
          CC     0.9745    0.9719    0.9732       748
          CM     0.9593    0.9571    0.9582       443
         DET     0.9690    0.9817    0.9753      1147
          FS     1.0000    0.9996    0.9998      2256
         JCV     0.7176    0.7825    0.7487       630
          JJ     0.8824    0.8401    0.8607      3652
         NCV     0.8020    0.8535    0.8270       949
         NDT     0.6154    0.5333    0.5714        15
         NIP     0.9601    0.9808    0.9703       884
         NNC     0.9368    0.9075    0.9219     12249
         NNJ     0.7047    0.8169    0.7567      1300
         NNP     0.9327    0.9459    0.9393      4806
         NUM     0.9433    0.9671    0.9551      1033
         NVB     0.6559    0.7871    0.7155       155
         NVF     0.0000    0.0000    0.0000         1
        POST     0.9427    

TrainOutput(global_step=2260, training_loss=0.35532055069914964, metrics={'train_runtime': 21688.0973, 'train_samples_per_second': 1.667, 'train_steps_per_second': 0.104, 'total_flos': 4725953826324480.0, 'train_loss': 0.35532055069914964, 'epoch': 4.0})

### Evaluate

In [8]:
trainer.evaluate()


d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

              precision    recall  f1-score   support

         ABB     0.9705    0.9809    0.9757       470
         AUX     0.9846    0.9726    0.9785       328
          CC     0.9745    0.9719    0.9732       748
          CM     0.9593    0.9571    0.9582       443
         DET     0.9690    0.9817    0.9753      1147
          FS     1.0000    0.9996    0.9998      2256
         JCV     0.7176    0.7825    0.7487       630
          JJ     0.8824    0.8401    0.8607      3652
         NCV     0.8020    0.8535    0.8270       949
         NDT     0.6154    0.5333    0.5714        15
         NIP     0.9601    0.9808    0.9703       884
         NNC     0.9368    0.9075    0.9219     12249
         NNJ     0.7047    0.8169    0.7567      1300
         NNP     0.9327    0.9459    0.9393      4806
         NUM     0.9433    0.9671    0.9551      1033
         NVB     0.6559    0.7871    0.7155       155
         NVF     0.0000    0.0000    0.0000         1
        POST     0.9427    

{'eval_loss': 0.27297502756118774,
 'eval_accuracy': 0.9235377741673436,
 'eval_f1': 0.9239023109313633,
 'eval_runtime': 112.2575,
 'eval_samples_per_second': 20.141,
 'eval_steps_per_second': 1.265,
 'epoch': 4.0}

In [9]:
from sklearn.metrics import classification_report

# Get predictions
predictions_output = trainer.predict(tokenized_dataset["test"])
preds = predictions_output.predictions.argmax(-1)
labels = predictions_output.label_ids

# Flatten predictions, ignoring -100
true_labels = []
predicted_labels = []

for pred_seq, label_seq in zip(preds, labels):
    for pred, label in zip(pred_seq, label_seq):
        if label != -100:
            true_labels.append(label)
            predicted_labels.append(pred)

# Dynamically detect used label IDs
used_label_ids = sorted(set(true_labels + predicted_labels))

# Match label IDs to tag names
target_names = [id2tag[i] for i in used_label_ids]

# Evaluate safely with matching label set
print(classification_report(
    true_labels,
    predicted_labels,
    labels=used_label_ids,
    target_names=target_names,
    zero_division=0  # prevents warnings for unseen labels
))


d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

              precision    recall  f1-score   support

         ABB     0.9705    0.9809    0.9757       470
         AUX     0.9846    0.9726    0.9785       328
          CC     0.9745    0.9719    0.9732       748
          CM     0.9593    0.9571    0.9582       443
         DET     0.9690    0.9817    0.9753      1147
          FS     1.0000    0.9996    0.9998      2256
         JCV     0.7176    0.7825    0.7487       630
          JJ     0.8824    0.8401    0.8607      3652
         NCV     0.8020    0.8535    0.8270       949
         NDT     0.6154    0.5333    0.5714        15
         NIP     0.9601    0.9808    0.9703       884
         NNC     0.9368    0.9075    0.9219     12249
         NNJ     0.7047    0.8169    0.7567      1300
         NNP     0.9327    0.9459    0.9393      4806
         NUM     0.9433    0.9671    0.9551      1033
         NVB     0.6559    0.7871    0.7155       155
         NVF     0.0000    0.0000    0.0000         1
        POST     0.9427    

In [10]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Step 1: Evaluate overall metrics using Hugging Face Trainer
eval_results = trainer.evaluate()
print("🔍 Evaluation Metrics from trainer.evaluate():")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Step 2: Run predictions
predictions_output = trainer.predict(tokenized_dataset["test"])
predictions = predictions_output.predictions
label_ids = predictions_output.label_ids

# Step 3: Get most likely tag indices
pred_labels = np.argmax(predictions, axis=2)

# Step 4: Prepare true and predicted tag names (flattened)
true_tags = []
predicted_tags = []

# Optional: Track mismatches with sentence-level tags
mismatches = []

for i in range(len(label_ids)):
    true_sent = []
    pred_sent = []
    for true_id, pred_id in zip(label_ids[i], pred_labels[i]):
        if true_id != -100:
            true_tag = id2tag[true_id]
            pred_tag = id2tag[pred_id]
            true_tags.append(true_tag)
            predicted_tags.append(pred_tag)
            true_sent.append(true_tag)
            pred_sent.append(pred_tag)
    
    if true_sent != pred_sent:
        mismatches.append((i, true_sent, pred_sent))

# Step 5: Classification report
unique_tags = sorted(list(set(true_tags + predicted_tags)))
print("\n📊 Classification Report:")
print(classification_report(true_tags, predicted_tags, labels=unique_tags, zero_division=0))

# Step 6: Confusion matrix
print("\n📉 Confusion Matrix (label indices):")
tag2id_filtered = {tag: idx for idx, tag in enumerate(unique_tags)}
y_true_ids = [tag2id_filtered[tag] for tag in true_tags]
y_pred_ids = [tag2id_filtered[tag] for tag in predicted_tags]
cm = confusion_matrix(y_true_ids, y_pred_ids)
print(cm)

# Step 7: Show a few mismatched sentences
print(f"\n🔎 Total mismatches found: {len(mismatches)}")
for idx, true_sent, pred_sent in mismatches[:5]:  # Limit to 5 for readability
    print(f"\nMISMATCHED SENTENCE {idx + 1}")
    print("True Tags :", true_sent)
    print("Pred Tags :", pred_sent)


d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

              precision    recall  f1-score   support

         ABB     0.9705    0.9809    0.9757       470
         AUX     0.9846    0.9726    0.9785       328
          CC     0.9745    0.9719    0.9732       748
          CM     0.9593    0.9571    0.9582       443
         DET     0.9690    0.9817    0.9753      1147
          FS     1.0000    0.9996    0.9998      2256
         JCV     0.7176    0.7825    0.7487       630
          JJ     0.8824    0.8401    0.8607      3652
         NCV     0.8020    0.8535    0.8270       949
         NDT     0.6154    0.5333    0.5714        15
         NIP     0.9601    0.9808    0.9703       884
         NNC     0.9368    0.9075    0.9219     12249
         NNJ     0.7047    0.8169    0.7567      1300
         NNP     0.9327    0.9459    0.9393      4806
         NUM     0.9433    0.9671    0.9551      1033
         NVB     0.6559    0.7871    0.7155       155
         NVF     0.0000    0.0000    0.0000         1
        POST     0.9427    

d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Acedemic\Level 4\research\RobeRta-POS\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

              precision    recall  f1-score   support

         ABB     0.9705    0.9809    0.9757       470
         AUX     0.9846    0.9726    0.9785       328
          CC     0.9745    0.9719    0.9732       748
          CM     0.9593    0.9571    0.9582       443
         DET     0.9690    0.9817    0.9753      1147
          FS     1.0000    0.9996    0.9998      2256
         JCV     0.7176    0.7825    0.7487       630
          JJ     0.8824    0.8401    0.8607      3652
         NCV     0.8020    0.8535    0.8270       949
         NDT     0.6154    0.5333    0.5714        15
         NIP     0.9601    0.9808    0.9703       884
         NNC     0.9368    0.9075    0.9219     12249
         NNJ     0.7047    0.8169    0.7567      1300
         NNP     0.9327    0.9459    0.9393      4806
         NUM     0.9433    0.9671    0.9551      1033
         NVB     0.6559    0.7871    0.7155       155
         NVF     0.0000    0.0000    0.0000         1
        POST     0.9427    

In [ ]:
from collections import Counter
import pandas as pd
from sklearn.metrics import classification_report

# --- Step 1: Extract tags from datasets ---
train_tags = [id2tag[tag_id] for sent in tokenized_dataset["train"]["labels"] for tag_id in sent if tag_id != -100]
test_tags = [id2tag[tag_id] for sent in tokenized_dataset["test"]["labels"] for tag_id in sent if tag_id != -100]

train_counter = Counter(train_tags)
test_counter = Counter(test_tags)

train_tag_set = set(train_counter.keys())
test_tag_set = set(test_counter.keys())

both_tags = sorted(train_tag_set & test_tag_set)
train_only_tags = sorted(train_tag_set - test_tag_set)
test_only_tags = sorted(test_tag_set - train_tag_set)

# --- Step 2: Get classification report as dict ---
# Ensure these lists are aligned with predictions
report = classification_report(true_tags, predicted_tags, output_dict=True, zero_division=0)

# --- Step 3: Combine all tags into rows ---
rows = []

# Helper function to pull metrics safely
def get_metric(tag, metric):
    return report[tag][metric] if tag in report else 0.0

# Tags in both sets
for tag in both_tags:
    total = train_counter[tag] + test_counter[tag]
    rows.append({
        "Tag": tag,
        "Train Count": train_counter[tag],
        "Test Count": test_counter[tag],
        "Total Count": total,
        "Tag Origin": "Both",
        "Precision": get_metric(tag, "precision"),
        "Recall": get_metric(tag, "recall"),
        "F1-Score": get_metric(tag, "f1-score")
    })

# Tags only in training
for tag in train_only_tags:
    rows.append({
        "Tag": tag,
        "Train Count": train_counter[tag],
        "Test Count": 0,
        "Total Count": train_counter[tag],
        "Tag Origin": "Train Only",
        "Precision": 0.0,
        "Recall": 0.0,
        "F1-Score": 0.0
    })

# Tags only in testing
for tag in test_only_tags:
    rows.append({
        "Tag": tag,
        "Train Count": 0,
        "Test Count": test_counter[tag],
        "Total Count": test_counter[tag],
        "Tag Origin": "Test Only",
        "Precision": get_metric(tag, "precision"),
        "Recall": get_metric(tag, "recall"),
        "F1-Score": get_metric(tag, "f1-score")
    })

# --- Step 4: Build DataFrame and sort within each category ---
df = pd.DataFrame(rows)

# Sort within each group
df = df.sort_values(by=["Tag Origin", "Precision", "Total Count"], ascending=[True, False, False]).reset_index(drop=True)

# --- Optional: Display or export ---
pd.set_option("display.max_rows", None)  # to see all rows
print(df)

# Optional export
#df.to_csv("tag_analysis_num-of-ep-4.csv", index=False)


      Tag  Train Count  Test Count  Total Count  Tag Origin  Precision  \
0      FS         9006        2256        11262        Both   1.000000   
1    PUNC         7117        1776         8893        Both   0.995506   
2     AUX         1232         328         1560        Both   0.984568   
3     PRP         5473        1413         6886        Both   0.981008   
4      CC         2929         748         3677        Both   0.974531   
5     ABB         1564         470         2034        Both   0.970526   
6     DET         4674        1147         5821        Both   0.969019   
7     NIP         3462         884         4346        Both   0.960133   
8      CM         1787         443         2230        Both   0.959276   
9      RP         4962        1172         6134        Both   0.954545   
10    UNK          312          43          355        Both   0.952381   
11    NUM         4413        1033         5446        Both   0.943343   
12   POST        14735        3647    

### Save and Use the Model

In [12]:
model.save_pretrained("sinhala-pos-xlm-r")
tokenizer.save_pretrained("sinhala-pos-xlm-r")


('sinhala-pos-xlm-r\\tokenizer_config.json',
 'sinhala-pos-xlm-r\\special_tokens_map.json',
 'sinhala-pos-xlm-r\\tokenizer.json')

In [13]:
from transformers import pipeline

pos_pipeline = pipeline("token-classification", model="sinhala-pos-xlm-r", tokenizer="sinhala-pos-xlm-r", aggregation_strategy="simple")

sentence = "මම පාසැල යමි"
tokens = sentence.split()  # Assuming simple whitespace tokenization
print(pos_pipeline(tokens))


Device set to use cuda:0


[[{'entity_group': 'PRP', 'score': np.float32(0.9918686), 'word': 'මම', 'start': 0, 'end': 2}], [{'entity_group': 'NNC', 'score': np.float32(0.828202), 'word': 'පාසැල', 'start': 0, 'end': 5}], [{'entity_group': 'VFM', 'score': np.float32(0.8610128), 'word': 'යමි', 'start': 0, 'end': 3}]]


In [14]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


PyTorch version: 2.7.1+cu118
CUDA available: True
CUDA device count: 1
GPU name: NVIDIA GeForce GTX 1050 Ti
